In [1]:
# imports
from IPython.display import clear_output
from PIL import Image
import os
import json
# clear
clear_output(wait=True)
# installing kaggle
!pip install -q kaggle
# user and key info
info = {"username":"jackbellamy","key":"3442898ba5e904ac3a48eb5b59398805"}
# opening file
json_object = json.dumps(info, indent=4)
with open("kaggle.json", "w") as outfile:
    outfile.write(json_object)
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# copying data to content file of kaggle which can be found on the left hand side of the screen
if not (os.path.exists("/content/nfl-big-data-bowl-2023")):
  !kaggle competitions download -c nfl-big-data-bowl-2023
  !mkdir nfl-big-data-bowl-2023
  !unzip nfl-big-data-bowl-2023.zip -d nfl-big-data-bowl-2023

 95% 164M/172M [00:01<00:00, 182MB/s]
100% 172M/172M [00:01<00:00, 145MB/s]
Archive:  nfl-big-data-bowl-2023.zip
  inflating: nfl-big-data-bowl-2023/games.csv  
  inflating: nfl-big-data-bowl-2023/pffScoutingData.csv  
  inflating: nfl-big-data-bowl-2023/players.csv  
  inflating: nfl-big-data-bowl-2023/plays.csv  
  inflating: nfl-big-data-bowl-2023/week1.csv  
  inflating: nfl-big-data-bowl-2023/week2.csv  
  inflating: nfl-big-data-bowl-2023/week3.csv  
  inflating: nfl-big-data-bowl-2023/week4.csv  
  inflating: nfl-big-data-bowl-2023/week5.csv  
  inflating: nfl-big-data-bowl-2023/week6.csv  
  inflating: nfl-big-data-bowl-2023/week7.csv  
  inflating: nfl-big-data-bowl-2023/week8.csv  


In [2]:
#import the libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.impute import SimpleImputer

In [3]:
#read csv files
dfPlays = pd.read_csv('/content//nfl-big-data-bowl-2023/plays.csv')
dfPlayer = pd.read_csv('/content/nfl-big-data-bowl-2023/players.csv')
dfWeek = pd.read_csv('/content/nfl-big-data-bowl-2023/week1.csv')

In [4]:
#function to convert gameClock time
def convertTimeToNumerical(t):
    t = t.split(":")
    if len(t) == 2:
        return int(t[0]) * 60 + int(t[1])
    return int(t[0]) * 3600 + int(t[1]) * 60 + int(t[2])
#function to convert players height
def heightInches(h):
    temp = list(map(int, h.split('-')))
    return temp[0]*12 + temp[1]

In [5]:
#converting values
dfPlays = pd.merge(dfWeek, dfPlays, on=['playId'])
dfPlays['gameClock_1'] = dfPlays['gameClock'].apply(convertTimeToNumerical)

def convertTimeToNumerical2(t):
    t = t.split(":")
    if len(t) == 2:
        return int(t[0])
    return int(t[0])

dfPlays['gameClock_new'] = dfPlays['gameClock'].apply(convertTimeToNumerical2)

In [15]:
# Selecting train and test data
xInput = dfPlays[['defendersInBox','quarter','yardsToGo','gameClock_new']]#,'gameClock']]#,'height','s']]

plays = dfPlays

C_data = plays.loc[plays['passResult']=="C"]
I_data = plays.loc[plays['passResult']=="I"]
IN_data = plays.loc[plays['passResult']=="IN"]
S_data = plays.loc[plays['passResult']=="S"]

all_data = pd.concat([C_data, I_data, IN_data, S_data], axis=0)

labels = np.array([0]*len(C_data)+[1]*len(I_data)+[2]*len(IN_data)+[3]*len(S_data))

xInput = all_data[['quarter','down','yardsToGo','absoluteYardlineNumber','gameClock_new']]

yInput = labels

xInput.head()

,quarter,down,yardsToGo,absoluteYardlineNumber,gameClock_new
2,1,1,10,72.0,13
4,1,3,4,41.0,13
5,1,3,2,77.0,13
8,1,1,10,72.0,13
10,1,3,4,41.0,13


In [25]:
#split data into train and test
#week1
x_train,x_test,y_train,y_test = train_test_split(xInput,yInput,test_size=0.25)
#scaling dataset
scaler = MinMaxScaler()
xTrainScaled = scaler.fit_transform(x_train)
xTestScaled = scaler.fit_transform(x_test)

print(xTestScaled)

[[0.25       0.25       0.33333333 0.75510204 0.13333333]
 [0.5        0.5        0.3        0.28571429 0.8       ]
 [0.5        0.5        0.23333333 0.93877551 0.4       ]
 ...
 [0.25       0.5        0.13333333 0.7755102  0.06666667]
 [0.5        0.25       0.33333333 0.2755102  0.13333333]
 [0.25       0.25       0.33333333 0.81632653 0.33333333]]


In [ ]:
#select model
model = DecisionTreeClassifier(max_depth = 25)
#train model
model.fit(xTrainScaled,y_train)

predictions = model.predict(xTrainScaled)
#observing performance metrics
accuracy = accuracy_score(y_train,predictions)*100
print("Accuracy of the model is {:.2f}%".format(accuracy))

predictions = model.predict(xTestScaled)
#observing performance metrics
accuracy = accuracy_score(y_test,predictions)*100
print("Accuracy of the model is {:.2f}%".format(accuracy))

print("-----------------")
print(y_test)
print(predictions)
print("-----------------")
#y_pred = np.argmax(y_test, axis=1)
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y_test, predictions)

print(confusion_matrix)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import to_categorical
import numpy as np
import cv2
from imutils import paths
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [10]:
plays = dfPlays

I_data = plays.loc[plays['passResult']=="I"]
C_data = plays.loc[plays['passResult']=="C"]
IN_data = plays.loc[plays['passResult']=="IN"]
S_data = plays.loc[plays['passResult']=="S"]


all_data = pd.concat([I_data, C_data, IN_data, S_data], axis=0)

all_data = pd.concat([I_data, C_data], axis=0)

data = all_data[['quarter','down','yardsToGo','absoluteYardlineNumber','gameClock_new']]

data = data.dropna(axis='rows',how='all')

# checking
data.head()

,quarter,down,yardsToGo,absoluteYardlineNumber,gameClock_new
0,1,3,2,43.0,13
3,1,3,4,41.0,14
6,1,3,2,43.0,13
9,1,3,4,41.0,14
12,1,3,2,43.0,13


In [11]:
features = np.array(data)

# pass outcome
labels = np.array([0]*len(I_data)+[1]*len(C_data)+[2]*len(IN_data)+[3]*len(S_data))
labels = np.array([0]*len(I_data)+[1]*len(C_data))
print("shape of features:", features.shape)
print("shape of labels:", labels.shape)
print("length of features:", len(features))
print("length of labels:", len(labels))

shape of features: (2893009, 5)
shape of labels: (2893009,)
length of features: 2893009
length of labels: 2893009


In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# Separate the test data
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

norm_y_test = y_test

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [13]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=features.shape[1]))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10)

Epoch 1/10
32139/63285 [==============>...............] - ETA: 1:00 - loss: 0.6403 - accuracy: 0.6446

KeyboardInterrupt: ignored

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn import metrics
y_pred1 = model.predict(x_test)


In [ ]:
y_pred = np.argmax(y_pred1, axis=1)

print("Predictions:")
print(y_pred)
print("Actual:")
print(norm_y_test)
print("")

confusion_matrix = metrics.confusion_matrix(norm_y_test, y_pred)

print(confusion_matrix)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()

data = history.history['accuracy']

fig.add_trace(go.Scatter(
    x=np.linspace(1,10,10),
    y=data,
    name="Accuracy",
    line=dict(color='black'),
    mode='lines',   
    #showlegend = True
))

fig.update_layout(
    width=600, 
    height=400,
    #xaxis_range=[],
    #yaxis_range=[],
    plot_bgcolor='white',
    xaxis_title="Epochs",
    yaxis_title="Accuracy",
    font=dict(
    family="Didone",
    size=18,  # Set the font size here
    color="Black",
    )
)

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
)

fig.update_layout(legend=dict(yanchor="bottom", y=0.7, xanchor="left", x=1.05))

fig.show()

#fig.write_image("/content/nn_acc.pdf")